<a href="https://colab.research.google.com/github/EmmanuelKiriinya/AfyaJamii/blob/main/maternal_health_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages
!pip install -q langchain
!pip install langchain-community langchain-groq faiss-cpu duckduckgo-search gtts googletrans speechrecognition pyttsx3 pydub transformers sentence-transformers ddgs

  Using cached langchain_community-0.3.30-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_groq-0.3.8-py3-none-any.whl.metadata (2.6 kB)
  Using cached faiss_cpu-1.12.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
  Using cached duckduckgo_search-8.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached gTTS-2.5.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached speechrecognition-3.14.3-py3-none-any.whl.metadata (30 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached groq-0.32.0-py3-none-any.whl.metadata (16 kB)
  Using cached primp-0.15.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# import os
# import speech_recognition as sr
# from gtts import gTTS
# import io
# from googletrans import Translator
# from langchain_groq import ChatGroq
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.tools import DuckDuckGoSearchRun
# from duckduckgo_search import DDGS
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain
# from langchain.schema import Document
# from langchain.prompts import PromptTemplate
# import requests
# from pydub import AudioSegment
# from pydub.playback import play
# import base64
# from IPython.display import Audio, display
# from google.colab import userdata

# os.environ["GROQ_API_KEY"] = userdata.get('api_key')

# class MaternalHealthRAG:
#     def __init__(self):
#         # Initialize components
#         self.llm = ChatGroq(
#             model_name="llama-3.1-8b-instant",
#             temperature=0.3
#         )

#         # Initialize embeddings
#         self.embeddings = HuggingFaceEmbeddings(
#             model_name="sentence-transformers/all-MiniLM-L6-v2"
#         )

#         # Initialize vector store
#         self.vector_store = None
#         self.retriever = None

#         # Initialize memory
#         self.memory = ConversationBufferMemory(
#             memory_key="chat_history",
#             return_messages=True,
#             output_key='answer'
#         )

#         # Initialize tools
#         self.search = DuckDuckGoSearchRun()
#         self.translator = Translator()

#         # Initialize speech recognition
#         self.recognizer = sr.Recognizer()

#         # Set up prompt template for maternal health
#         self.prompt_template = PromptTemplate(
#             input_variables=['context', 'question', 'chat_history', 'user_name'],
#             template="""
#             You are a compassionate maternal health assistant. Address the user as {user_name} when appropriate. Provide accurate, supportive,
#             and actionable advice for pregnancy and maternal health.

#             Context from latest research: {context}

#             Conversation history: {chat_history}

#             Current question: {question}

#             Please provide:
#             1. Evidence-based advice tailored to the mother's situation
#             2. Emotional support and reassurance
#             3. Specific actionable steps
#             4. When appropriate, suggest consulting healthcare providers

#             Answer:
#             """
#         )

#         # Initialize with basic maternal health knowledge
#         self.initialize_knowledge_base()

#     def initialize_knowledge_base(self):
#         """Initialize with basic maternal health documents"""
#         basic_docs = [
#             Document(page_content="""
#             Nutrition during pregnancy:
#             - Folic acid: 400-800 mcg daily to prevent neural tube defects
#             - Iron: 27 mg daily to support increased blood volume
#             - Calcium: 1000 mg daily for bone development
#             - Protein: 71g daily for tissue growth
#             - Stay hydrated with 8-10 glasses of water daily
#             """),
#             Document(page_content="""
#             Emotional well-being in pregnancy:
#             - Practice stress-reduction techniques like meditation
#             - Maintain social connections for emotional support
#             - Get adequate sleep (7-9 hours per night)
#             - Communicate openly with partner and healthcare providers
#             - Seek professional help for persistent mood changes
#             """),
#             Document(page_content="""
#             Common pregnancy symptoms and management:
#             - Morning sickness: Eat small, frequent meals; ginger tea may help
#             - Fatigue: Rest when needed; light exercise can boost energy
#             - Back pain: Practice good posture; prenatal yoga can help
#             - Always consult healthcare provider about concerning symptoms
#             """),
#             Document(page_content="""
#             Healthy Kenyan meals during pregnancy:
#             - Ugali with leafy greens (sukuma wiki, collard greens) and lean protein (fish, beans)
#             - Githeri (mixture of maize and beans) for fiber and protein
#             - Sweet potatoes and yams for complex carbohydrates and vitamins
#             - Fruits like mangoes, avocados, and oranges for vitamins and minerals
#             - Dairy products like milk and yogurt for calcium
#             """)
#         ]

#         self.vector_store = FAISS.from_documents(basic_docs, self.embeddings)
#         self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})

#         # Set up conversation chain
#         self.qa_chain = ConversationalRetrievalChain.from_llm(
#             llm=self.llm,
#             retriever=self.retriever,
#             memory=self.memory,
#             combine_docs_chain_kwargs={"prompt": self.prompt_template}
#         )

#     def search_latest_information(self, query):
#         """Search for latest maternal health information"""
#         try:
#             search_query = f"site:who.int OR site:cdc.gov OR site:acog.org {query} pregnancy maternal health 2024"
#             results = self.search.run(search_query)
#             return results
#         except Exception as e:
#             return f"Search error: {str(e)}"

#     def update_knowledge_base(self, new_info):
#         """Update vector store with new information"""
#         new_doc = Document(page_content=new_info)
#         self.vector_store.add_documents([new_doc])

#     def get_response(self, question, user_name="user", use_latest_search=True):
#         """Generate response with optional internet search"""
#         try:
#             # If needed, search for latest information
#             if use_latest_search:
#                 latest_info = self.search_latest_information(question)
#                 if latest_info and "error" not in latest_info.lower():
#                     self.update_knowledge_base(f"Latest research: {latest_info}")

#             # Get response from QA chain
#             response = self.qa_chain({"question": question, "user_name": user_name})
#             return response['answer']

#         except Exception as e:
#             return f"Error generating response: {str(e)}"

#     def translate_text(self, text, target_language='en'):
#         """Translate text to target language"""
#         try:
#             translation = self.translator.translate(text, dest=target_language)
#             return translation.text
#         except:
#             return text  # Return original if translation fails

#     def text_to_speech(self, text, language='en'):
#         """Convert text to speech and return as IPython.display.Audio"""
#         try:
#             # Use gTTS for online TTS
#             tts = gTTS(text=text, lang=language, slow=False)
#             # Save to a BytesIO object
#             audio_bytes = io.BytesIO()
#             tts.write_to_fp(audio_bytes)
#             audio_bytes.seek(0)

#             # Return as IPython.display.Audio
#             return Audio(audio_bytes.read(), rate=24000)

#         except Exception as e:
#             print(f"Error generating speech: {e}")
#             return None


#     def speech_to_text(self, audio_source):
#         """Convert speech to text from an audio file path"""
#         try:
#             # Use the provided audio file path
#             with sr.AudioFile(audio_source) as source:
#                 audio = self.recognizer.record(source)

#             text = self.recognizer.recognize_google(audio)
#             return text
#         except sr.UnknownValueError:
#             return "Could not understand audio"
#         except sr.RequestError as e:
#             return f"Error with speech recognition: {e}"
#         except FileNotFoundError:
#              return f"Audio file not found at {audio_source}"
#         except Exception as e:
#             return f"An unexpected error occurred during speech recognition: {e}"


#     def process_query(self, query, user_name="user", input_type='text', target_language='en', use_latest_search=True):
#         """Main method to process user queries"""
#         # Handle voice input
#         if input_type == 'voice' and isinstance(query, str) and query == '':
#              return "Please provide audio input.", "", ""

#         # Translate query to English for processing if needed
#         if target_language != 'en':
#             query_en = self.translate_text(query, 'en')
#         else:
#             query_en = query

#         # Get response
#         response_en = self.get_response(query_en, user_name=user_name, use_latest_search=use_latest_search)

#         # Translate response to target language
#         if target_language != 'en':
#             response = self.translate_text(response_en, target_language)
#         else:
#             response = response_en

#         return response, query_en, response_en

# # Initialize the system
# maternal_assistant = MaternalHealthRAG()

In [3]:
# import gradio as gr
# import os

# # Define the chat function for Gradio's ChatInterface
# def maternal_health_chat(message, history):
#     """
#     Chat function to interact with the MaternalHealthRAG class for Gradio ChatInterface.
#     Handles text input and provides text output.
#     """
#     # Process the text message
#     response, _, _ = maternal_assistant.process_query(
#         message,
#         input_type='text',  # ChatInterface provides text input
#         target_language='en', # Assuming English for chat input, can add language dropdown later
#         use_latest_search=True # Use latest search by default for chat
#     )
#     return response

# # Create the Gradio ChatInterface
# chat_interface = gr.ChatInterface(
#     fn=maternal_health_chat,
#     chatbot=gr.Chatbot(height=400),
#     textbox=gr.Textbox(placeholder="Ask a question about maternal health...", container=False, scale=7),
#     title="🤰 Maternal Health Assistant",
#     description="Ask questions about maternal health during pregnancy.",
#     theme="soft",
#     examples=[
#         "What are the most important nutrients during pregnancy?",
#         "How can I cope with morning sickness?",
#         "Is it safe to exercise while pregnant?"
#     ],
#     cache_examples=True,
# )

# # Launch the interface
# chat_interface.launch(debug=True)

In [1]:
!pip install -q langchain langchain-community langchain-groq faiss-cpu duckduckgo-search gradio speechrecognition gtts googletrans==4.0.0-rc1 pydub transformers sentence-transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in re

KeyboardInterrupt: 

In [ ]:


import os
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import io
from googletrans import Translator
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document
from langchain.prompts import PromptTemplate
import base64
import tempfile
from pydub import AudioSegment
import numpy as np

# Get Groq API key (you'll need to set this in Colab secrets)
try:
    from google.colab import userdata
    os.environ["GROQ_API_KEY"] = userdata.get('api_key')
except:
    # For local development or if using input
    print("invalid_key")
class MaternalHealthRAG:
    def __init__(self):
        # Initialize components
        self.llm = ChatGroq(
            model_name="llama-3.1-8b-instant",
            temperature=0.3,
            max_tokens=1024
        )

        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # Initialize vector store
        self.vector_store = None
        self.retriever = None

        # Initialize memory
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key='answer'
        )

        # Initialize tools
        self.search = DuckDuckGoSearchRun()
        self.translator = Translator()

        # Initialize speech recognition
        self.recognizer = sr.Recognizer()

        # User session management
        self.user_sessions = {}

        # Set up prompt template for maternal health
        self.prompt_template = PromptTemplate(
            input_variables=['context', 'question', 'chat_history'],
            template="""
            ⚠️ MEDICAL DISCLAIMER: You are an AI assistant providing general information only.
            Always recommend consulting healthcare professionals for medical advice.

            You are a compassionate maternal health assistant focused on Kenyan women.
            Provide accurate, supportive, and actionable advice for pregnancy and maternal health.

            Context from latest research: {context}

            Conversation history: {chat_history}

            Current question: {question}

            Please provide:
            1. Evidence-based advice tailored to Kenyan context
            2. Emotional support and reassurance
            3. Specific actionable steps using locally available resources
            4. Cultural appropriate recommendations
            5. When appropriate, suggest consulting healthcare providers

            Focus on:
            - Nutrition using Kenyan foods (sukuma wiki, ndengu, ugali, etc.)
            - Emotional well-being and stress management
            - Pregnancy risk factors and warning signs
            - Local healthcare resources in Kenya

            Answer in a warm, supportive tone:
            """
        )

        # Initialize with basic maternal health knowledge
        self.initialize_knowledge_base()

    def get_user_session(self, user_id):
        """Get or create user session"""
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = {
                'memory': ConversationBufferMemory(
                    memory_key="chat_history",
                    return_messages=True,
                    output_key='answer'
                ),
                'language': 'en',
                'name': 'User'
            }
        return self.user_sessions[user_id]

    def initialize_knowledge_base(self):
        """Initialize with Kenyan-focused maternal health documents"""
        kenyan_docs = [
            Document(page_content="""
            Kenyan Pregnancy Nutrition Guide:
            Essential Nutrients for Kenyan Diet:
            - Iron-rich foods: Sukuma wiki (kale), ndengu (lentils), liver, red meat
            - Folate sources: Fresh leafy greens, beans, citrus fruits, avocado
            - Calcium: Maziwa (milk), yogurt, ndizi (bananas), sesame seeds
            - Protein: Kuku (chicken), samaki (fish), eggs, beans, omena
            - Traditional beneficial foods: Uji (porridge), mchuzi wa nazi

            Kenyan Food Safety:
            - Wash all fruits and vegetables thoroughly
            - Cook meat and eggs completely
            - Avoid street food during pregnancy
            - Drink boiled or purified water
            - Pasteurized dairy products only
            """),
            Document(page_content="""
            Emotional Well-being for Kenyan Mothers:
            - Practice stress-reduction: Deep breathing, gentle walking
            - Maintain social connections with family and community
            - Get adequate sleep (7-9 hours per night)
            - Communicate openly with partner and healthcare providers
            - Seek professional help for persistent mood changes
            - Traditional support systems: Family gatherings, church community

            Kenyan Healthcare Resources:
            - Linda Mama Program: Free maternity services
            - Antenatal Care: Minimum 4 visits recommended
            - Emergency contacts: 112 or 999
            - Nairobi Women's Hospital: +254 703 082 000
            """),
            Document(page_content="""
            Pregnancy Warning Signs in Kenya:
            URGENT MEDICAL ATTENTION NEEDED FOR:
            - Severe headache with vision changes (pre-eclampsia)
            - Vaginal bleeding or fluid leakage
            - Severe abdominal pain
            - High fever (38°C or above)
            - Reduced baby movement after 28 weeks
            - Swelling of hands, face, or feet

            Common Pregnancy Symptoms Management:
            - Morning sickness: Eat small, frequent meals; ginger tea
            - Fatigue: Rest when needed; light walking
            - Back pain: Good posture; prenatal exercises
            - Heartburn: Small meals; avoid spicy foods
            """),
            Document(page_content="""
            Lactation Nutrition in Kenya:
            - Increase calorie intake by 500 calories daily
            - Traditional lactation foods: Chicken soup, uji fortified with nuts
            - Hydration: Drink water before and after breastfeeding
            - Local superfoods: Moringa leaves, amaranth, pumpkin leaves
            - Continue prenatal vitamins if recommended

            Kenyan Cultural Practices:
            - 40-day rest period after delivery
            - Special foods for milk production
            - Family support during postpartum period
            - Traditional birth attendants in some communities
            """)
        ]

        self.vector_store = FAISS.from_documents(kenyan_docs, self.embeddings)
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})

    def search_latest_information(self, query):
        """Search for latest maternal health information with Kenyan focus"""
        try:
            kenya_query = f"{query} Kenya pregnancy maternal health 2024"
            results = self.search.run(kenya_query)
            return results
        except Exception as e:
            return f"Search error: {str(e)}"

    def update_knowledge_base(self, new_info):
        """Update vector store with new information"""
        if new_info and len(new_info) > 50:  # Only add substantial content
            new_doc = Document(page_content=f"Latest information: {new_info}")
            self.vector_store.add_documents([new_doc])

    def get_response(self, question, user_id="default", use_latest_search=True):
        """Generate response with optional internet search"""
        try:
            session = self.get_user_session(user_id)

            # If needed, search for latest information
            latest_info = ""
            if use_latest_search:
                latest_info = self.search_latest_information(question)
                if latest_info and "error" not in latest_info.lower():
                    self.update_knowledge_base(latest_info)

            # Create conversation chain with user-specific memory
            qa_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.retriever,
                memory=session['memory'],
                combine_docs_chain_kwargs={"prompt": self.prompt_template}
            )

            # Get response
            response = qa_chain({"question": question})
            return response['answer']

        except Exception as e:
            return f"Error generating response: {str(e)}"

    def translate_text(self, text, target_language='en'):
        """Translate text to target language"""
        try:
            if target_language == 'en':
                return text
            translation = self.translator.translate(text, dest=target_language)
            return translation.text
        except Exception as e:
            print(f"Translation error: {e}")
            return text  # Return original if translation fails

    def text_to_speech(self, text, language='en'):
        """Convert text to speech and return audio file path"""
        try:
            # Map language codes to gTTS codes
            lang_map = {'en': 'en', 'sw': 'sw', 'fr': 'fr', 'es': 'es'}
            tts_lang = lang_map.get(language, 'en')

            # Create temporary file
            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
                tts = gTTS(text=text, lang=tts_lang, slow=False)
                tts.save(tmp_file.name)
                return tmp_file.name
        except Exception as e:
            print(f"TTS error: {e}")
            return None

    def speech_to_text(self, audio_file):
        """Convert speech to text from audio file"""
        try:
            with sr.AudioFile(audio_file) as source:
                audio = self.recognizer.record(source)

            # Try multiple languages
            try:
                text = self.recognizer.recognize_google(audio, language='en-US')
                return text, 'en'
            except:
                try:
                    text = self.recognizer.recognize_google(audio, language='sw-TZ')
                    return text, 'sw'
                except:
                    return "Could not understand audio", 'en'

        except Exception as e:
            return f"Speech recognition error: {str(e)}", 'en'

    def process_query(self, query, user_id="default", input_type='text',
                     target_language='en', use_latest_search=True):
        """Main method to process user queries"""
        session = self.get_user_session(user_id)

        # Handle voice input
        if input_type == 'voice':
            if hasattr(query, 'name'):  # It's a file
                text, detected_lang = self.speech_to_text(query)
                session['language'] = detected_lang
            else:
                text = query
        else:
            text = query

        # Translate query to English for processing if needed
        if session['language'] != 'en':
            query_en = self.translate_text(text, 'en')
        else:
            query_en = text

        # Get response
        response_en = self.get_response(query_en, user_id, use_latest_search)

        # Translate response to target language
        if target_language != 'en':
            response = self.translate_text(response_en, target_language)
        else:
            response = response_en

        # Generate audio
        audio_file = self.text_to_speech(response, target_language)

        return response, audio_file, query_en

# Initialize the system
maternal_assistant = MaternalHealthRAG()

# Create Gradio interface
def chat_interface(message, history, language, use_search, audio_input, user_id):
    """Gradio chat interface function"""

    input_type = 'voice' if audio_input is not None else 'text'
    query = audio_input if audio_input is not None else message

    if not query:
        return history, "", None

    # Process query
    response, audio_file, _ = maternal_assistant.process_query(
        query=query,
        user_id=user_id,
        input_type=input_type,
        target_language=language,
        use_latest_search=use_search
    )

    # Add to history
    history.append([query if input_type == 'text' else "[Voice message]", response])

    # Return audio file for playback
    audio_output = audio_file if audio_file else None

    return history, "", audio_output

def clear_chat(user_id):
    """Clear chat history for user"""
    if user_id in maternal_assistant.user_sessions:
        maternal_assistant.user_sessions[user_id]['memory'].clear()
    return [], "", None

# Create Gradio blocks
with gr.Blocks(theme=gr.themes.Soft(), title="🤰 Mama Afya Advisor - Kenyan Maternal Health") as demo:
    gr.Markdown("""
    # 🤰 Mama Afya Advisor
    ## Kenyan Maternal Health & Nutrition Assistant

    ⚠️ **Medical Disclaimer**: This AI provides general health information only.
    Always consult qualified healthcare professionals for medical advice.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            user_id = gr.Textbox(
                label="User ID",
                value="user_001",
                info="Enter a unique ID to maintain conversation history"
            )
            language = gr.Dropdown(
                choices=["en", "sw", "fr", "es"],
                value="en",
                label="Response Language",
                info="Choose language for responses"
            )
            use_search = gr.Checkbox(
                label="Use Internet Search",
                value=True,
                info="Search for latest information"
            )
            clear_btn = gr.Button("Clear Chat History", variant="secondary")

            gr.Markdown("""
            ### Emergency Contacts (Kenya):
            - **General Emergency**: 112 or 999
            - **Nairobi Women's Hospital**: +254 703 082 000
            - **Aga Khan Hospital**: +254 20 366 2000
            """)

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                label="Mama Afya Conversation",
                height=400,
                show_copy_button=True
            )

            with gr.Row():
                text_input = gr.Textbox(
                    placeholder="Type your question about maternal health...",
                    label="Text Input",
                    scale=4
                )
                audio_input = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    label="Voice Input",
                    scale=1
                )

            with gr.Row():
                submit_btn = gr.Button("Send Message", variant="primary")
                audio_output = gr.Audio(
                    label="Voice Response",
                    interactive=False,
                    visible=True
                )

    # Examples
    gr.Examples(
        examples=[
            ["What Kenyan foods are rich in iron during pregnancy?", "en"],
            ["Ni vyakula gani vya Kenya vinavyokuwa na chuma wakati wa ujauzito?", "sw"],
            ["What are warning signs during pregnancy?", "en"],
            ["How much water should I drink daily?", "en"],
            ["Nutrition for breastfeeding mothers in Kenya", "en"]
        ],
        inputs=[text_input, language]
    )

    # Event handlers
    submit_btn.click(
        fn=chat_interface,
        inputs=[text_input, chatbot, language, use_search, audio_input, user_id],
        outputs=[chatbot, text_input, audio_output]
    )

    text_input.submit(
        fn=chat_interface,
        inputs=[text_input, chatbot, language, use_search, audio_input, user_id],
        outputs=[chatbot, text_input, audio_output]
    )

    clear_btn.click(
        fn=clear_chat,
        inputs=[user_id],
        outputs=[chatbot, text_input, audio_output]
    )

# Launch the interface
if __name__ == "__main__":
    demo.launch(
        share=True,  # Creates public link
        debug=True,
        show_error=True
    )

/tmp/ipython-input-2777411774.py:380: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2d7b4bd355991c58f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
